In [59]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %load_ext autoreload
%autoreload 2

In [60]:
from utils.core import Load_data
import pandas as pd
from Model.preprocessing.pipelines import Data_Process
from Model.preprocessing.datasetManager import DatasetManger
import tensorflow as tf
from Model.base.evaluator import EvalutionClassifer, Drow
import os
import numpy as np
import shap

In [61]:
model_path = "./trained_model/10model0.keras"
dataset_pd = dataset_pd = pd.read_csv(
    "../dataset/FullDatasetAndWavlet.csv", index_col="annot_id")

dataset_pd.loc["125_"] = dataset_pd.loc["125_"].fillna(0.9)

column_interest = [
    'ti-rads_level',
    'original_ngtdm_Coarseness',
    'wavelet-HH_glcm_MaximumProbability',
    'wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis',
    'sex',
    'age',
    'size_x',
    'ti-rads_margin',
    'ti-rads_echogenicfoci',
    'wavelet-LH_glcm_SumEntropy',
    'original_firstorder_Kurtosis',
    'wavelet-LL_glszm_LargeAreaLowGrayLevelEmphasis',
    'wavelet-HL_firstorder_Energy',
    'size_y',
    'ti-rads_shape',
    'wavelet-HH_glrlm_RunLengthNonUniformityNormalized',
    'size_z',
    'wavelet-LL_glcm_MCC',
    'wavelet-HH_firstorder_Variance',
    'wavelet-HL_glszm_HighGrayLevelZoneEmphasis',
    'wavelet-HH_glszm_LargeAreaHighGrayLevelEmphasis',
    'original_firstorder_InterquartileRange',
    'wavelet-HL_glrlm_ShortRunLowGrayLevelEmphasis',
    'wavelet-HL_glszm_LargeAreaHighGrayLevelEmphasis',
    'original_firstorder_Skewness',
    'wavelet-LH_gldm_DependenceEntropy',
    'original_glszm_LargeAreaLowGrayLevelEmphasis',
    'wavelet-HH_glrlm_RunLengthNonUniformity',
    'original_shape2D_Sphericity',
    'wavelet-HL_firstorder_Median'
]

X = dataset_pd[column_interest]


Y = dataset_pd['histopath_diagnosis'].astype(np.float32)

batch_size = 32
test_size = 0.1



managerdataset = DatasetManger(X, Y, batch_size=batch_size, n_splits=2, test_size=test_size)

categorical_features = ["sex"]  # remvoed by mrmr "location"

numerical_features = ["ti-rads_level", "age"]

Ti_rads_featurea = ["ti-rads_margin", "ti-rads_shape", "ti-rads_echogenicfoci"]

numerical_features = list(X[X.columns[~X.columns.isin(categorical_features + Ti_rads_featurea)]].columns)

scaler = Data_Process(X, categorical_features,  numerical_features, Ti_rads_featurea)
X = scaler.fit_transform()

METRICS = [
    tf.keras.metrics.BinaryAccuracy,
    tf.keras.metrics.Precision,
    tf.keras.metrics.Recall,
]

In [62]:
model = tf.keras.models.load_model(model_path)
shap.initjs()


In [63]:
def predict(x):
    df = pd.DataFrame(x, columns=list(X.columns))
    df = {key:value for key, value in df.items()}
    print(df)
    return model.predict(df).flatten()

In [64]:
explainer = shap.KernelExplainer(predict, X[0:1000])

{'categorical__sex_Female': 0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Name: categorical__sex_Female, Length: 1000, dtype: float64, 'categorical__sex_Male': 0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
995    0.0
996    0.0
997    0.0
998    0.0
999    0.0
Name: categorical__sex_Male, Length: 1000, dtype: float64, 'num__ti-rads_level': 0     -0.990852
1     -0.990852
2     -0.990852
3     -0.990852
4     -0.990852
         ...   
995   -2.152807
996   -2.152807
997   -2.152807
998   -2.152807
999   -2.152807
Name: num__ti-rads_level, Length: 1000, dtype: float64, 'num__original_ngtdm_Coarseness': 0     -0.212457
1     -0.212457
2     -0.212457
3     -0.212457
4     -0.212457
         ...   
995   -0.212457
996   -0.212457
997   -0.212457
998   -0.212457
999   -0.212457
Name: num__original_ngtdm_Coarseness, Length: 1000, dtype: float64, 'num__wavelet-HH_glcm_MaximumProbability': 0      0.2

Using 1000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [65]:
shape_value = explainer.shap_values(X[1000:2000])

  0%|          | 0/1000 [00:00<?, ?it/s]

{'categorical__sex_Female': 0    1.0
Name: categorical__sex_Female, dtype: float64, 'categorical__sex_Male': 0    0.0
Name: categorical__sex_Male, dtype: float64, 'num__ti-rads_level': 0   -2.152807
Name: num__ti-rads_level, dtype: float64, 'num__original_ngtdm_Coarseness': 0   -0.212457
Name: num__original_ngtdm_Coarseness, dtype: float64, 'num__wavelet-HH_glcm_MaximumProbability': 0   -0.095259
Name: num__wavelet-HH_glcm_MaximumProbability, dtype: float64, 'num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis': 0   -0.320714
Name: num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis, dtype: float64, 'num__age': 0   -0.814402
Name: num__age, dtype: float64, 'num__size_x': 0   -0.137329
Name: num__size_x, dtype: float64, 'num__wavelet-LH_glcm_SumEntropy': 0    0.299531
Name: num__wavelet-LH_glcm_SumEntropy, dtype: float64, 'num__original_firstorder_Kurtosis': 0   -0.619783
Name: num__original_firstorder_Kurtosis, dtype: float64, 'num__wavelet-LL_glszm_LargeAreaLowGrayLevelEmphasis': 0 

  0%|          | 1/1000 [02:57<49:13:27, 177.39s/it]

{'categorical__sex_Female': 0    1.0
Name: categorical__sex_Female, dtype: float64, 'categorical__sex_Male': 0    0.0
Name: categorical__sex_Male, dtype: float64, 'num__ti-rads_level': 0   -2.152807
Name: num__ti-rads_level, dtype: float64, 'num__original_ngtdm_Coarseness': 0   -0.212457
Name: num__original_ngtdm_Coarseness, dtype: float64, 'num__wavelet-HH_glcm_MaximumProbability': 0   -0.051296
Name: num__wavelet-HH_glcm_MaximumProbability, dtype: float64, 'num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis': 0   -0.208573
Name: num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis, dtype: float64, 'num__age': 0   -0.814402
Name: num__age, dtype: float64, 'num__size_x': 0   -0.137329
Name: num__size_x, dtype: float64, 'num__wavelet-LH_glcm_SumEntropy': 0    0.472058
Name: num__wavelet-LH_glcm_SumEntropy, dtype: float64, 'num__original_firstorder_Kurtosis': 0   -0.81449
Name: num__original_firstorder_Kurtosis, dtype: float64, 'num__wavelet-LL_glszm_LargeAreaLowGrayLevelEmphasis': 0  

  0%|          | 2/1000 [53:45<517:15:28, 1865.86s/it]

...

{'categorical__sex_Female': 0    1.0
Name: categorical__sex_Female, dtype: float64, 'categorical__sex_Male': 0    0.0
Name: categorical__sex_Male, dtype: float64, 'num__ti-rads_level': 0   -2.152807
Name: num__ti-rads_level, dtype: float64, 'num__original_ngtdm_Coarseness': 0   -0.212457
Name: num__original_ngtdm_Coarseness, dtype: float64, 'num__wavelet-HH_glcm_MaximumProbability': 0   -0.131759
Name: num__wavelet-HH_glcm_MaximumProbability, dtype: float64, 'num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis': 0   -0.580884
Name: num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis, dtype: float64, 'num__age': 0   -0.814402
Name: num__age, dtype: float64, 'num__size_x': 0   -0.137329
Name: num__size_x, dtype: float64, 'num__wavelet-LH_glcm_SumEntropy': 0    0.491111
Name: num__wavelet-LH_glcm_SumEntropy, dtype: float64, 'num__original_firstorder_Kurtosis': 0   -0.948326
Name: num__original_firstorder_Kurtosis, dtype: float64, 'num__wavelet-LL_glszm_LargeAreaLowGrayLevelEmphasis': 0 

  1%|          | 11/1000 [1:30:26<78:48:45, 286.88s/it]

{'categorical__sex_Female': 0    1.0
Name: categorical__sex_Female, dtype: float64, 'categorical__sex_Male': 0    0.0
Name: categorical__sex_Male, dtype: float64, 'num__ti-rads_level': 0   -2.152807
Name: num__ti-rads_level, dtype: float64, 'num__original_ngtdm_Coarseness': 0   -0.212457
Name: num__original_ngtdm_Coarseness, dtype: float64, 'num__wavelet-HH_glcm_MaximumProbability': 0   -0.153804
Name: num__wavelet-HH_glcm_MaximumProbability, dtype: float64, 'num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis': 0   -0.294916
Name: num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis, dtype: float64, 'num__age': 0   -0.814402
Name: num__age, dtype: float64, 'num__size_x': 0   -0.137329
Name: num__size_x, dtype: float64, 'num__wavelet-LH_glcm_SumEntropy': 0    0.371265
Name: num__wavelet-LH_glcm_SumEntropy, dtype: float64, 'num__original_firstorder_Kurtosis': 0   -0.890796
Name: num__original_firstorder_Kurtosis, dtype: float64, 'num__wavelet-LL_glszm_LargeAreaLowGrayLevelEmphasis': 0 

  1%|          | 11/1000 [1:30:31<135:38:36, 493.75s/it]


KeyboardInterrupt: 

In [ ]:
shape_value

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.00497945],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.00610674],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.00561989]])

In [ ]:
shap.force_plot(explainer.expected_value, shape_value,
                X.iloc[100:200, :])

In [ ]:
X.iloc[200, :]

categorical__sex_Female                                   1.000000
categorical__sex_Male                                     0.000000
num__ti-rads_level                                       -0.990852
num__original_ngtdm_Coarseness                           -0.212454
num__wavelet-HH_glcm_MaximumProbability                   0.098754
num__wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis     -0.824711
num__age                                                 -2.456971
num__size_x                                              -0.460047
num__wavelet-LH_glcm_SumEntropy                          -1.792895
num__original_firstorder_Kurtosis                         0.835028
num__wavelet-LL_glszm_LargeAreaLowGrayLevelEmphasis       0.651701
num__wavelet-HL_firstorder_Energy                        -0.864328
num__size_y                                               0.248120
num__wavelet-HH_glrlm_RunLengthNonUniformityNormalized    1.341551
num__size_z                                              -0.69

In [ ]:
shap.force_plot(explainer.expected_value, shape_value, X.iloc[160, 0:23])

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed